In [1]:

#construction of RNN
'''
batch_size = 1
seq_len = 3
input_size = 4
hidden_size = 2
num_layers = 1

cell = torch.nn.RNN(input_size=input_size, hidden_size=hidden_size,num_layers=num_layers) #rnn实现
#补充：batch_first=True 交换seqlen和batchsize，方便构造数据集

# (seqLen, batchSize, inputSize)
inputs = torch.randn(seq_len, batch_size, input_size) #3,1,4-----x1,x2,x3三种特征，1批次，4天的x1(或x2,x3)数据
#例二 hello->ohlol seq=5,batchsize=1,inputsize=4 单词长度5个字母，单词出现4个字母，每个字母给出数字代表
# e=0 h=1 l=2 o=3 则hello转化为[1,0,2,2,3],再转化为矩阵(one-hot encoding)
'''
[0,1,0,0]
[1,0,0,0]
[0,0,1,0]
[0,0,1,0]
[0,0,0,1]
'''
#此时out也是一个【1,4】向量，例如[0.8,0.5,0.2,0.1]，进行softmax-NLLLoss-one-hot(crossentropy)


hidden = torch.zeros(num_layers, batch_size, hidden_size) #单个rnn cell中rnn层数，批次，2个隐藏层数据

out, hidden = cell(inputs, hidden)

print('Output size:', out.shape) 
print('Output:', out)
print('Hidden size: ', hidden.shape)
print('Hidden: ', hidden)
'''

"\n#此时out也是一个【1,4】向量，例如[0.8,0.5,0.2,0.1]，进行softmax-NLLLoss-one-hot(crossentropy)\n\n\nhidden = torch.zeros(num_layers, batch_size, hidden_size) #单个rnn cell中rnn层数，批次，2个隐藏层数据\n\nout, hidden = cell(inputs, hidden)\n\nprint('Output size:', out.shape) \nprint('Output:', out)\nprint('Hidden size: ', hidden.shape)\nprint('Hidden: ', hidden)\n"

In [2]:
#hello-ohlol实战

import torch
batch_size = 1
input_size = 4
hidden_size = 4

#01 prepare dataset
idx2char = ['e', 'h', 'l', 'o']
x_data = [1, 0, 2, 2, 3]
y_data = [3, 1, 2, 3, 2]
one_hot_lookup = [[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 1, 0],
[0, 0, 0, 1]]
x_one_hot = [one_hot_lookup[x] for x in x_data] #x_one_hot在此处是一个one_hot列表，5*4
inputs = torch.Tensor(x_one_hot).view(-1, batch_size, input_size)
labels = torch.LongTensor(y_data).view(-1, 1) #-1维度为seqlen

print(x_one_hot)

[[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 0, 1]]


In [3]:
#02 design model
class Model(torch.nn.Module):
    def __init__(self, input_size, hidden_size, batch_size):
        super(Model, self).__init__()
        self.batch_size = batch_size
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.rnncell = torch.nn.RNNCell(input_size=self.input_size, hidden_size=self.hidden_size)
        
    def forward(self, input, hidden):                  
        hidden = self.rnncell(input, hidden)
        return hidden
    
    def init_hidden(self):                               #h0初始化
        return torch.zeros(self.batch_size, self.hidden_size)
    
net = Model(input_size, hidden_size, batch_size)

In [4]:
#03 loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.1)

In [5]:
#04 training cycle
for epoch in range(15):
    loss = 0
    optimizer.zero_grad()
    hidden = net.init_hidden()
    print('Predicted string: ', end='')
    for input, label in zip(inputs, labels): #inputs和labels配对，input[batchsize,inputsize],而inputs[seqlen,batchsize,inputsize]
        hidden = net(input, hidden)
        loss += criterion(hidden, label)    #不可加.item，因为RNN是循环神经网络，一直在构建计算图
        _, idx = hidden.max(dim=1)          #hidden[batchsize,hidden_size],取每个batch中hidden_size中最大值
        print(idx2char[idx.item()], end='')
    loss.backward()
    optimizer.step()
    print(', Epoch [%d/15] loss=%.4f' % (epoch+1, loss.item()))

Predicted string: eoooe, Epoch [1/15] loss=7.3975
Predicted string: lllll, Epoch [2/15] loss=6.2772
Predicted string: lllll, Epoch [3/15] loss=5.7591
Predicted string: lhlll, Epoch [4/15] loss=5.2099
Predicted string: lhlll, Epoch [5/15] loss=4.6142
Predicted string: lhlll, Epoch [6/15] loss=4.0999
Predicted string: ohlol, Epoch [7/15] loss=3.7622
Predicted string: ohlol, Epoch [8/15] loss=3.5307
Predicted string: ohlol, Epoch [9/15] loss=3.3202
Predicted string: ohlol, Epoch [10/15] loss=3.0891
Predicted string: ohlol, Epoch [11/15] loss=2.8509
Predicted string: ohlol, Epoch [12/15] loss=2.6570
Predicted string: ohlol, Epoch [13/15] loss=2.5214
Predicted string: ohlol, Epoch [14/15] loss=2.4218
Predicted string: ohlol, Epoch [15/15] loss=2.3411


In [ ]:
#05 改进

'''
one-hot缺点
1 The one-hot vectors are high-dimension. 例如10000个英语单词，则需要10000维
2 The one-hot vectors are sparse. 所有点集中在坐标轴，【，，，，】中只有一个为1，其他全0
3 The one-hot vectors are hardcode 硬编码，非学习

使用embedding 